In [251]:
import pandas as pd

In [252]:
y = pd.read_csv('nightlife_10002_1_yelp.csv')
yNew = y[['id', 'name', 'review_count', 'categories', 'rating', 'coordinates', 'location', 'rev1', 'rev2', 'rev3']].copy()

In [253]:
#create categories index

cat = pd.read_json('categories.json')

cat['aliasTitle'] = '{' + cat['alias'] + ', ' + cat['title']+ '}'
catIndex = pd.Index(list(cat['aliasTitle']))

In [254]:
#get number of categories
yNew['numCats'] = yNew.categories.str.count('}, {') + 1

#clean up for parsing
yNew['catIndexes'] = yNew['categories'].str.replace('\'', '')
yNew['catIndexes'] = yNew['catIndexes'].str.replace('[', '')
yNew['catIndexes'] = yNew['catIndexes'].str.replace(']', '')
yNew['catIndexes'] = yNew['catIndexes'].str.replace('alias: ', '')
yNew['catIndexes'] = yNew['catIndexes'].str.replace('title: ', '')

In [255]:
#match categories with dictionary index

for x in yNew['catIndexes']:
    for y in catIndex: 
        if y in x:
            yNew['catIndexes'] = yNew['catIndexes'].str.replace(y, str(catIndex.get_loc(y)))
yNew.head(2)

,id,name,review_count,categories,rating,coordinates,location,rev1,rev2,rev3,numCats,catIndexes
0,a3O2KzK6rh8VZNq1jhz81Q,The Delancey,586,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",3.0,"{'latitude': 40.71803, 'longitude': -73.98528}","{'address1': '168 Delancey St', 'address2': ''...",I went on a Saturday night with some girlfrien...,- Right by the Williamsburg bridge so it's fur...,3 stars for venue\r\n3.5 stars for music.\r\n\...,2,"828, 371"
1,iM3zYjjmnJ06VVmxvTo9eQ,Nitecap,339,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 40.719125, 'longitude': -73.985732}","{'address1': '151 Rivington St', 'address2': '...",Came here on a Saturday night with a friend. G...,My friends and I went to this place last week ...,Very cool bar hidden around the corner. Went t...,1,305


In [256]:
#extract lat long from coordinates

yNew['latitude'] = yNew.coordinates.str.extract('latitude\': ([^/]*,)')
yNew['latitude'] = (yNew['latitude'].str.replace(',','')).astype(float)
yNew['longitude'] = yNew.coordinates.str.extract('longitude\': ([^/]*})')
yNew['longitude'] = (yNew['longitude'].str.replace('}','')).astype(float)

In [257]:
yNew['fullAddress'] = yNew.location.str.extract('display_address\': \[([^/]*\')')
yNew['fullAddress'] = yNew['fullAddress'].str.replace("\'","")

In [258]:
#zip code col
yNew['zipcode'] = (yNew.location.str.extract(r'zip_code\': \'(\d+)')).astype(int)

In [259]:
yfinal = yNew[['id', 'name', 'fullAddress', 'zipcode', 'review_count', 'rating', 'numCats', 'catIndexes', 'rev1', 'rev2', 'rev3','latitude', 'longitude']].copy()

In [260]:
yfinal

,id,name,fullAddress,zipcode,review_count,rating,numCats,catIndexes,rev1,rev2,rev3,latitude,longitude
0,a3O2KzK6rh8VZNq1jhz81Q,The Delancey,"168 Delancey St, New York, NY 10002",10002,586,3.0,2,"828, 371",I went on a Saturday night with some girlfrien...,- Right by the Williamsburg bridge so it's fur...,3 stars for venue\r\n3.5 stars for music.\r\n\...,40.718030,-73.985280
1,iM3zYjjmnJ06VVmxvTo9eQ,Nitecap,"151 Rivington St, New York, NY 10002",10002,339,4.0,1,305,Came here on a Saturday night with a friend. G...,My friends and I went to this place last week ...,Very cool bar hidden around the corner. Went t...,40.719125,-73.985732
2,aiRctIzCj9J3UqBHd-Z3HQ,Please Don't Tell,"113 St Marks Pl, New York, NY 10009",10009,1801,4.0,2,"828, 1310",I went to this famous speakeasy when I first m...,Great low key place to have a few drinks. I sa...,PDT is a wonderful cocktail bar. The atmosphe...,40.727098,-73.983763
3,H7S7-vF2z_HfaLo6BGPetw,Attaboy,"134 Eldridge St, New York, NY 10002",10002,551,4.0,2,"305, 1310",Attaboy was the Aladdin to my Jasmine because ...,I had such high expectations for Attaboy as ev...,Attaboy is always packed. \r\nI came here on a...,40.718887,-73.991335
4,yjmedpxGM2RlCRbOafOU7Q,Rumpus Room,"249 Eldridge St, New York, NY 10002",10002,85,3.5,1,371,Best bar ever! Great drinks great music! Fun f...,In regards to all the people complaining of 15...,Nice and roomy at 11:30. Quick bad service. Th...,40.722972,-73.989657
5,X0f9UYHoE4OxnuUq801vMg,The Upstairs at 66,"46 Stone St, New York, NY 10004",10004,15,5.0,3,"828, 55, 1310",This place is absolutely wonderful. I threw a ...,We came on a Friday night and we didn't have t...,"I've been coming to Stone Street for a while, ...",40.704180,-74.010480
6,xDELCKWk0sjaSFw8CeQkPA,Club Cumming,"505 E 6th St, New York, NY 10009",10009,41,4.0,2,"371, 828",I've been to a few drag cabaret shows here (fe...,"Ok, I feel terrible because I want to give it ...",Club Cumming is a East Village gay bar backed ...,40.725350,-73.983420
7,DfGVOGrlTQtkx2zdNM-Zbw,Mr. Purple,"180 Orchard St, Hotel Indigo, New York, NY 10002",10002,665,3.0,2,"828, 305",Wonderful time with a group of friends in the ...,great spot to hangout with your friends \r\nev...,Security is always on edge. They refuse to hel...,40.721750,-73.988020
8,fzN8PuCQ1l2sAvCyiMWC8g,Old Man Hustle,"39 Essex St, New York, NY 10002",10002,182,4.5,3,"305, 405, 314",Random walk by and met Rachel is the comic edg...,You don't come here for the speciality drinks ...,There are very few places in the city I enjoy ...,40.716267,-73.989698
9,4nZlMrDw-mrvmfEVT-cQrw,The Village Underground,"130 W 3rd St, New York, NY 10012",10012,312,4.0,3,"371, 828, 314",I was so excited to see comedy in NYC! The bes...,Had a great time here. Reserved my ticket onli...,Highly recommend - we had a great time here! I...,40.730694,-74.000923
